In [0]:
!pip3 install torchvision
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, RNN, LSTM
from torch.nn.functional import softmax, relu, nll_loss, log_softmax

from sklearn.manifold import TSNE
import h5py

    100% |████████████████████████████████| 61kB 4.3MB/s 
    100% |████████████████████████████████| 519.5MB 32kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x590e4000 @  0x7fe1412662a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 2.0MB 5.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [0]:
# Load data
with h5py.File('/content/drive/My Drive/Colab Notebooks/L4M_dataset.hdf5', 'r') as hf:
    data = hf['data'][:]
    label = hf['label'][:]

In [0]:
print(np.max(data[:,1]))

17769


In [0]:
seq = [] #Initalize
lab = []
current_user = -1
for i in range(data.shape[0]): # going through the data and sorting into user sequences
    if data[i,0]!=current_user:
        current_user = data[i,0]
        seq.append([data[i,1]])
        lab.append([label[i]])        
    else:
        seq[-1].append(data[i,1]) # new user line
        lab[-1].append(label[i])
        
train_idx = np.random.choice(len(seq),(len(seq)*8)//10,replace = False)
seq_train = [seq[i] for i in train_idx]
seq_test = [seq[i] for i in range(len(seq)) if i not in train_idx]
lab_train = [lab[i] for i in train_idx]
lab_test = [lab[i] for i in range(len(lab)) if i not in train_idx]

seq_train.sort(key=lambda x:len(x))
lab_train.sort(key=lambda x:len(x))
seq_test.sort(key=lambda x:len(x))
lab_test.sort(key=lambda x:len(x))
seq_train = [item for item in seq_train if len(item) > 19]
lab_train = [item for item in lab_train if len(item) > 19]
seq_test = [item for item in seq_test if len(item) > 19]
lab_test = [item for item in lab_test if len(item) > 19]
seq_length_train = [len(x) for x in seq_train]
seq_length_test = [len(x) for x in seq_test]

print(len(seq_train))
print(len(seq_test))
print(len(lab_train))
print(len(lab_test))

120445
30071
120445
30071


In [0]:
# Define network
output_size = 17770
num_input = 17770
hidden_size_rnn1 = 1000
hidden_size_lin1 = 1000

hidden_size_rnn2 = 5000
hidden_size_lin2 = 5000

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.one_hot = nn.Embedding(num_input, num_input)
        self.one_hot.weight.data = torch.eye(num_input)
        self.one_hot.weight.detach_()
        
        self.rnn1 = LSTM(input_size=num_input,
                        hidden_size=hidden_size_rnn1,
                        num_layers=1,
                        bidirectional=False,
                        batch_first = True)
        
        self.lin1 = Linear(in_features=hidden_size_rnn1,
                           out_features=hidden_size_lin1,
                           bias=False)
        
        #self.rnn2 = LSTM(input_size=hidden_size_lin1,
        #                 hidden_size=hidden_size_rnn2,
        #                 num_layers=1,
        #                 bidirectional=False,
        #                 batch_first=True)
        
        #self.lin2 = Linear(in_features=hidden_size_rnn2,
        #                   out_features=hidden_size_lin2,
        #                   bias=False)
        
        self.l_out = Linear(in_features=hidden_size_lin1,
                            out_features=output_size,
                            bias=False)
        
        self.criteria = nn.CrossEntropyLoss()
    
    def forward(self, x):
        out = {}
        x = self.one_hot(x)
        
        x, hn = self.rnn1(x)
        
        x = relu(self.lin1(x))
        
        
        #x, hn = self.rnn2(x)
        
        #x = relu(self.lin2(x))
        
        out = self.l_out(x)
        return out
    
net = Net()
if use_cuda:
    print('##converting network to cuda-enabled')
    net.cuda()
print(net)

##converting network to cuda-enabled
Net(
  (one_hot): Embedding(17770, 17770)
  (rnn1): LSTM(17770, 1000, batch_first=True)
  (lin1): Linear(in_features=1000, out_features=1000, bias=False)
  (l_out): Linear(in_features=1000, out_features=17770, bias=False)
  (criteria): CrossEntropyLoss()
)


In [0]:
{p[0]: p[1].requires_grad for p in net.named_parameters()}

{'l_out.weight': True,
 'lin1.weight': True,
 'one_hot.weight': False,
 'rnn1.bias_hh_l0': True,
 'rnn1.bias_ih_l0': True,
 'rnn1.weight_hh_l0': True,
 'rnn1.weight_ih_l0': True}

In [0]:
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [0]:
# Train and validation loop
batch_size = 50
net.train()
counter = 0
temp = 0
for i in np.random.choice(len(seq_train)//batch_size,len(seq_train)//batch_size, replace=False):
    batch = seq_train[i*batch_size:(i+1)*batch_size]
    batch_length = seq_length_train[i*batch_size:(i+1)*batch_size]
    batch_label = lab_train[i*batch_size:(i+1)*batch_size]
    
    # Cutting
    min_len_batch = min(batch_length)
    #batch_cut = np.zeros((batch_size,min_len_batch))
    #label_cut = np.zeros((batch_size,min_len_batch))
    #for j in range(batch_size):
    batch = np.array([item[:min_len_batch] for item in batch]) 
    batch_label = np.array([item[:min_len_batch] for item in batch_label])
    batch = torch.from_numpy(batch).long()
    batch_label = torch.from_numpy(batch_label).long()
    if use_cuda:
        batch = Variable(batch.cuda())
        batch_label = Variable(batch_label.cuda())
    output = net(batch)
    #print("MAX",output.max())
    #print('batch_length:',batch_length)
    batch_loss = net.criteria(output.view(-1,output_size),batch_label.view(-1))
    temp = temp + batch_loss
    if counter%50==0:
      print('Train completed:',counter)
      print('Loss'+' '+str(counter)+':',temp/50)
      temp = 0
    counter += 1
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

Train completed: 0
Loss 0: tensor(0.1957, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 50
Loss 50: tensor(8.9428, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 100
Loss 100: tensor(7.9159, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 150
Loss 150: tensor(7.8275, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 200
Loss 200: tensor(7.5522, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 250
Loss 250: tensor(7.3570, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 300
Loss 300: tensor(7.1244, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 350
Loss 350: tensor(6.9800, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 400
Loss 400: tensor(6.7727, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 450
Loss 450: tensor(6.6104, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 500
Loss 500: tensor(6.5469, device='cuda:0', grad_fn=<DivBackward0>)
Train completed: 550
Loss 550: tensor(6.4567, dev

In [0]:
net.eval()
count = 0
good = 0
with torch.no_grad():
  for user in seq_test:
      inputs = torch.from_numpy(np.array(user[:-5],dtype=float)).long()
      if use_cuda:
          inputs = Variable(inputs.cuda())
      labels = user[-5:]
      inputs = inputs.view(1,inputs.size()[0])
      outputs = net(inputs)
      out = softmax(outputs[:,-1,:],dim=1).cpu().numpy().reshape(-1)
      #print(outputs.min(),outputs.max(),outputs.sum())
      #print(np.argmax(outputs))
      top20 = np.argsort(out)[-20:]
      okay = np.any([movie in labels for movie in top20])
      if count%1000==0:
        print(count)
      count += 1
      if okay:
        good +=1
print(good/count)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
0.5865451764158159
